In [6]:
import os
import noisereduce as nr  
from pathlib import Path
import torchaudio
import torch
import numpy as np

In [13]:

filedirpath = Path("./wav/").resolve()
outdirpath = Path("./stage1/").resolve()

filenames = sorted(os.listdir(filedirpath))

for filename in filenames:
    filepath = filedirpath.joinpath(filename)

    data, fs = torchaudio.load(str(filepath))
    data_np = np.mean(data.numpy(), axis=0)
    data_np = data_np / np.max(data_np)
    resampled = torchaudio.functional.resample(torch.from_numpy(data_np)[None, :], fs, 32000)
    torchaudio.save(outdirpath.joinpath(filename), resampled, 32000)

In [15]:
indir = Path('./resampled/')
outdir = Path('./chunks/')

SR = 32000
SECONDS_PER_SAMPLE = 30

# for classname in classnames:
# classdir = indir.joinpath(classname)

classdata = torch.tensor([], dtype=torch.float32)
filenames = sorted(os.listdir(indir))

current_class = None
for filename in filenames:
    classname = filename[:filename.index('-', filename.index('-') + 1)]
    
    if current_class != classname:
        if current_class is not None:
            chunks = torch.split(classdata, split_size_or_sections=SR*SECONDS_PER_SAMPLE, dim=1)[:-1]
            for i, chunk in enumerate(chunks):
                outpath = outdir.joinpath(f'{current_class}_{i}.wav')

                torchaudio.save(str(outpath), chunk, SR)


        current_class = classname
    
    data, sr = torchaudio.load(str(indir.joinpath(filename)))
    assert(sr == SR)
    assert(data.dtype == torch.float32)
    classdata = torch.cat((classdata, data), dim=1)



In [ ]:
n_fft = 2048
hop_length = n_fft // 4

fig, ax = plt.subplots(2, 1)

data = None # TODO

def myspecshow(data, ax):
    spec = np.abs(lr.stft(data, n_fft=n_fft, hop_length=hop_length))**2
    mspec = lr.feature.melspectrogram(sr=fs, S=spec)

    mspec = lr.power_to_db(mspec, ref=np.max)
    return mspec, specshow(mspec, sr=fs, y_axis='mel', x_axis='time', hop_length=hop_length, ax=ax)

data_np = np.mean(data.numpy(), axis=0)
print(data_np.shape)
data_np = data_np / np.max(data_np)
data_filt = nr.reduce_noise(data_np, fs)
torchaudio.save('filtered.wav', torch.from_numpy(data_filt[None, :]), fs)

spec, img = myspecshow(data_filt, ax[0])
fig.colorbar(img, ax=ax, format="%+2.f dB")

o_env = lr.onset.onset_strength(sr=fs, S=spec)
times = lr.times_like(o_env, sr=fs)
onsets = lr.onset.onset_detect(onset_envelope=o_env, sr=fs)
print(onsets[:10])


ax[1].vlines(times[onsets], 0, o_env.max(), color='r', alpha=0.9)
ax[1].plot(times, o_env)
ax[1].set_xlim(times[0], times[-1])



In [108]:
import os

for file in os.listdir("chunks"):
    data, fs = torchaudio.load('chunks/' + file)

    resampled = torchaudio.functional.resample(data, fs, 32000)

    print(file)
    torchaudio.save('chunks32k/' + file, resampled, 32000)

    

A112.wav
A42.wav
A52.wav
B38.wav
B61.wav
A104.wav
A14.wav
B190.wav
B195.wav
A182.wav
B0.wav
B9.wav
B6.wav
B85.wav
B140.wav
A198.wav
A179.wav
B90.wav
B163.wav
B176.wav
A217.wav
A32.wav
B59.wav
B139.wav
B196.wav
A170.wav
A16.wav
B27.wav
B68.wav
B112.wav
A48.wav
A51.wav
A24.wav
B15.wav
A33.wav
A195.wav
A211.wav
B129.wav
B83.wav
B117.wav
B199.wav
A37.wav
B22.wav
A156.wav
B194.wav
A208.wav
B42.wav
A163.wav
B103.wav
A210.wav
A81.wav
A66.wav
B41.wav
A94.wav
A68.wav
B137.wav
A72.wav
B162.wav
A63.wav
B57.wav
A181.wav
A178.wav
B144.wav
B76.wav
A26.wav
B160.wav
B161.wav
A147.wav
B202.wav
B185.wav
A100.wav
A98.wav
B201.wav
A157.wav
B178.wav
A86.wav
A109.wav
A95.wav
A9.wav
A159.wav
A45.wav
B203.wav
B147.wav
B16.wav
A129.wav
A185.wav
B8.wav
A183.wav
B146.wav
A154.wav
A3.wav
A191.wav
A2.wav
B97.wav
A205.wav
B87.wav
A74.wav
B50.wav
A143.wav
A114.wav
B205.wav
B158.wav
A201.wav
B14.wav
A146.wav
A144.wav
B166.wav
A87.wav
A186.wav
A200.wav
A153.wav
B107.wav
B165.wav
A44.wav
A139.wav
B20.wav
A15.wav
A145.w

In [ ]:

for file in os.listdir("chunks"):
    data, fs = torchaudio.load('chunks/' + file)
    assert(fs == 32000)
    spec = torchaudio.transforms.MelSpectrogram(sample_rate=32000, n_fft=1024)